<a href="https://colab.research.google.com/github/uroojfatima18/AI-Agents/blob/main/Context_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#local context

In [2]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.4/720.4 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00


In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from pydantic  import BaseModel
from agents import AsyncOpenAI,OpenAIChatCompletionsModel,RunConfig
from google.colab import userdata


In [5]:
gemini_api_key = userdata.get("GEMINI_API_KEY")

In [6]:
# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

In [7]:
from agents import set_default_openai_client, set_tracing_disabled
set_default_openai_client(external_client)
set_tracing_disabled(True)

In [8]:

import asyncio
from dataclasses import dataclass
from agents import Agent, RunContextWrapper, Runner, function_tool

@dataclass
class UserInfo:      #object using dataclass
  name: str
  age: int
  location:str ="Pakistan"

@function_tool
async def fetch_user_age(Wrapper: RunContextWrapper[UserInfo]) ->str:
  '''Returns the age of the user.'''
  print("[Tool]" , Wrapper, "\n\n")
  return f" This User {Wrapper.context.name} and {Wrapper.context.age}year's old."


@function_tool
async def fetch_user_location(Wrapper: RunContextWrapper[UserInfo]) ->str:
  '''Returns the location of the user.'''
  print("[Tool current location ]" , Wrapper, "\n\n")
  return f"The User{Wrapper.context.name} is from {Wrapper.context.location} "

@function_tool
async def fetch_user_name(Wrapper: RunContextWrapper[UserInfo]) ->str:
  '''Returns the name of the user.'''
  print("[Tool]" , Wrapper, "\n\n")
  return f"The User{Wrapper.context.name} is from {Wrapper.context.name} "

async def main():
  user_info = UserInfo(name="Urooj", age =19)

  agent = Agent[UserInfo](
        name="Assistant",
        tools=[fetch_user_location,fetch_user_age,fetch_user_name],
        model=model
        )
  result = await Runner.run(
        starting_agent=agent,
        input="What is the name , age , current location of user?",
        context=user_info,
    )
  print(result.final_output)

if __name__ == "__main__":
  asyncio.run(main())


[Tool] RunContextWrapper(context=UserInfo(name='Urooj', age=19, location='Pakistan'), usage=Usage(requests=1, input_tokens=48, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=15, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=63)) 


[Tool] RunContextWrapper(context=UserInfo(name='Urooj', age=19, location='Pakistan'), usage=Usage(requests=1, input_tokens=48, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=15, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=63)) 


[Tool current location ] RunContextWrapper(context=UserInfo(name='Urooj', age=19, location='Pakistan'), usage=Usage(requests=1, input_tokens=48, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=15, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=63)) 


The user's name is Urooj, age is 19, and current location is Pakistan.



#Agent/LLM Context

In [10]:
from agents import enable_verbose_stdout_logging
enable_verbose_stdout_logging()

In [11]:
import asyncio
from dataclasses import dataclass
from agents import Agent, RunContextWrapper, Runner, function_tool

@dataclass
class UserInfo:      #object using dataclass
  name: str
  age: int
  location:str ="Pakistan"

@function_tool
async def Greet_user(context:RunContextWrapper[UserInfo],greeting :str)->str:
  """greet with user with thier name.
   Args:
    greeting: A specialed greeting message for user
  """
  name = context.context.name
  return f"{name} ,{greeting}"



async def main():
  userinfo=UserInfo(name="Urooj fatima" ,age ="19")

  agent = Agent[UserInfo](
        name="Assistant",
        tools=[Greet_user],
        model=model,
        instructions="Always greet the user using greet_user and warmly greet them panaversity. "
        )

  result = await Runner.run(
        starting_agent=agent,
        input="hello",
        context=userinfo,
    )
  print(result.final_output)


if __name__ == "__main__":
  asyncio.run(main())




Tracing is disabled. Not creating trace Agent workflow
Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op
Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7c11028b7e30>
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7c11028b7e30>


...

LLM resp:
{
  "content": "Hello Urooj fatima, Welcome! I hope you are having a great day! How can I help you today?\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}

LLM resp:
{
  "content": "Hello Urooj fatima, Welcome! I hope you are having a great day! How can I help you today?\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "Hello Urooj fatima, Welcome! I hope you are having a great day! How can I help you today?\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace
Resetting current trace


DEBUG:openai.agents:Resetting current trace


Hello Urooj fatima, Welcome! I hope you are having a great day! How can I help you today?

